In [1]:
# Corps fini et courbe de Montgomery : By^2 = x(x^2 + Ax + 1)
K = GF(101)
A = K(3)
#B = K(1)

E = EllipticCurve(K, [0, A, 0, 1, 0])  # y^2 = x(x^2 + Ax + 1)

R.<X> = PolynomialRing(K)

def direct_poly(P, Q):
    return (X - (P + Q)[0]) * (X - (P - Q)[0])

def kummer_poly(xP, xQ, A):
    trace = -2*((xP*xQ + 1)*(xP + xQ) + 2*A*xP*xQ) / ((xP - xQ)^2)
    product = ((xP*xQ - 1)^2) / ((xP - xQ)^2)
    return X^2 + trace*X + product

In [3]:
def pgcd_polynomes(f, g, verbose=False):
    """
    Calcule manuellement le PGCD de deux polynômes f et g
    en utilisant l'algorithme d'Euclide, sans utiliser .gcd().
    
    Entrées :
        - f, g : polynômes dans un même anneau (par exemple R.<X> = PolynomialRing(QQ))
        - verbose (optionnel) : si True, affiche les étapes de l'algorithme
        
    Sortie :
        - Le PGCD de f et g
    """
    
    if f.parent() != g.parent():
        raise ValueError("Les deux polynômes doivent appartenir au même anneau.")
    
    while not g.is_zero():
        q, r = f.quo_rem(g)  # Division euclidienne
        if verbose:
            print(f"Division : {f} = ({q}) * ({g}) + ({r})")
        f, g = g, r  # Mettre à jour
        
    # À la fin, f est le PGCD
    return f.monic()  # Optionnel : rendre le PGCD monique (coefficient dominant 1)



def addition_compatible(P1, P2, T, A):
    # Coordonnées affines
    xP1 = P1[0]
    xP2 = P2[0]
    xT  = T[0]

    # Calcul de P2 + T et P1 + T
    P2_plus_T = P2 + T
    xP2_plus_T = P2_plus_T[0]

    P1_plus_T = P1 + T
    xP1_plus_T = P1_plus_T[0]

    # Calcul des deux polynômes
    f1 = kummer_poly(xP1, xP2_plus_T, A)
    f2 = kummer_poly(xP2, xP1_plus_T, A)

    # PGCD des deux polynômes
    pgcd_polynomes(f1, f2, verbose=True)

    return g

# Exécution d'un exemple:


# Calcul
G = addition_compatible(P1, P2, T, A)

print("PGCD obtenu :", G)

# Récupération de la racine (x(P1 + P2 + T))
if G.degree() == 1:
    coeffs = G.coefficients(sparse=False)
    racine = -coeffs[0]/coeffs[1]
    print("x(P1 + P2 + T) =", racine)
else:
    print("Erreur : PGCD de degré inattendu :", G.degree())
    


NameError: name 'P1' is not defined

In [10]:

def xADD_inverse_affine(xP, xQ, xPplusQ):
    """
    Addition différentielle version affine.
    À partir de x(P), x(Q), x(P+Q),
    retourne x(P-Q).
    """
    numerator = (xP * xQ - 1)^2 * xPplusQ
    denominator = (xP - xQ)^2
    if denominator == 0:
        raise ZeroDivisionError("Division par zéro dans xADD_inverse_affine")
    return numerator / denominator

# 🧪 Exemple d'utilisation :

# Choix de P2 et T aléatoires
while True:
    P2 = E.random_point()
    T = E.random_point()
    if P2 != E(0) and T != E(0) and not (2*T == E(0)):
        break

# Coordonnées affines
xP2 = P2[0]
xT = T[0]

# x(P2 + T)
P2_plus_T = P2 + T
xP2plusT = P2_plus_T[0]

# Calcul de x(P2 - T)
x_P2_minus_T = xADD_inverse_affine(xP2, xT, xP2plusT)

print("x(P2 - T) =", x_P2_minus_T)


x(P2 - T) = 25


In [11]:
def addition_compatible(P1, P2, T, A):
    """
    Fonction addition compatible :
    - Calcule kummer_poly(x(P1), x(P2))
    - Calcule kummer_poly(x(P2-T), x(P1+T))
    - Fait leur PGCD
    - Retourne le PGCD (qui aura pour racine x(P1+P2))
    """
    # Coordonnées affines
    xP1 = P1[0]
    xP2 = P2[0]
    xT  = T[0]

    # x(P1 + T)
    P1_plus_T = P1 + T
    xP1plusT = P1_plus_T[0]

    # x(P2 + T)
    P2_plus_T = P2 + T
    xP2plusT = P2_plus_T[0]

    # Calcul de x(P2 - T) en utilisant addition différentielle inverse
    xP2minusT = xADD_inverse_affine(xP2, xT, xP2plusT)

    # Calcul des deux polynômes
    f1 = kummer_poly(xP1, xP2, A)
    f2 = kummer_poly(xP2minusT, xP1plusT, A)

    # PGCD des deux polynômes
    g = f1.gcd(f2)

    return g

# 🧪 Exemple d'utilisation :

# Choix de P1, P2 et T aléatoires
while True:
    P1 = E.random_point()
    P2 = E.random_point()
    T = E.random_point()
    if P1 != -P2 and P1 != P2 and P1 != E(0) and P2 != E(0) and T != E(0) and not (2*T == E(0)):
        break

G = addition_compatible(P1, P2, T, A)

print("PGCD obtenu :", G)

# Récupération de la racine (x(P1 + P2))
if G.degree() == 1:
    coeffs = G.coefficients(sparse=False)
    racine = -coeffs[0]/coeffs[1]
    print("x(P1 + P2) =", racine)
else:
    print("Erreur : PGCD de degré inattendu :", G.degree())


PGCD obtenu : 1
Erreur : PGCD de degré inattendu : 0
